# Blue Brain BioExplorer
![](../../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer
from mediamaker import MovieMaker
import os

hostname = 'localhost:5000'
resource_folder = '../../../tests/test_files/'
resource_folder = '/home/favreau/git/BioExplorer/bioexplorer/pythonsdk/tests/test_files/'
    
be = BioExplorer(hostname)
core = be.core_api()
mm = MovieMaker(be)
version = be.version()
print('Version ' + version)

sequence_name = 'low_glucose'

In [ ]:
'''Reset current scene'''
status = be.reset()

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

In [ ]:
cache_folder = resource_folder + 'caches/' + version + '/'

In [ ]:
cache_filename = cache_folder + sequence_name + '_scenario.bioexplorer'
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 152:
    print('ERROR: Not all models have been loaded!!! Expected 152, found ' + str(nb_models))
else:
    print('All good!')

In [ ]:
'''Show loaded models'''
status = be.set_models_visibility(True)

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=1, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.exposure = 1.0
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5000
params.shadows = 1.0
params.soft_shadows = 0.02
params.fog_start = 1000
params.fog_thickness = 300
params.max_bounces = 1
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

status = core.clear_lights()
status = core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-0.7,-0.4,-1],
    intensity=1.0, is_visible=False
)

In [ ]:
status = be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    specular_exponent=50.0
)

In [ ]:
for model in core.scene.models:
    model_id = model['id']
    model_name = model['name']
    if be.NAME_COLLAGEN in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        emissions = list()
        for i in range(nb_materials):
            palette.append([1,1,1])
            emissions.append(0.2)
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
            emissions=emissions
        )        
    if "Emile" in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)-1
        print(model_name)
        palette = list()
        a = 0.25
        for i in range(nb_materials):
            palette.append([a * 0.78,a * 0.47,a * 0.34])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
        )
        
status = core.scene.commit()

## Snapshots

In [ ]:
k = 4
image_size = [k*960,k*540]
image_spp = 64
image_format = 'png'
aperture_multiplier = 0.0

dof = ''
if aperture_multiplier != 0.0:
    dof = 'dof_'
    image_format='jpg'

core.set_camera(current='bio_explorer_perspective')

output_folder = resource_folder + 'frames/' + \
                version + '/' + sequence_name + '/'

output_suffix = '_' + dof + str(image_size[0]) + 'x' + \
                str(image_size[1]) + '_' + str(image_spp) + 'spp.png'

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder
if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

print(output_folder)
print(output_suffix)

In [ ]:
''' Virus overview '''
status =  mm.set_camera(
    direction = [0.0, 0.0, -1.0],
    origin = [-10, 10, 180],
    up = [0.0, 1.0, 0.0]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'virus_overview' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' Spike '''
status = mm.set_camera(
    direction = [0.0, 0.0, -1.0],
    origin = [-5, 75, 10],
    up = [0.0, 1.0, 0.0]
)

In [ ]:
mm.create_snapshot(
    size=image_size, 
    path=output_folder + 'spike' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' Protein M et E '''
status = mm.set_camera(
    direction = [-0.950, 0.226, 0.211],
    origin = [3.863, 84.170, 60.234],
    up = [0.233, 0.972, 0.004]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'proteins_m_and_e' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' Lactoferrin on virus'''
status = mm.set_camera(
    direction = [0.646, -0.067, -0.760],
    origin = [-72.177, 14.070, 79.730],
    up = [0.071, 0.997, -0.027]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'lactoferrin_on_virus' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' Overview SP'''
status = mm.set_camera(
    direction = [-0.406, -0.133, -0.903],
    origin = [201.631, 61.669, 302.864],
    up = [0.008, 0.988, -0.149]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'surfactant_overview' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' Zoom SPD'''
status = mm.set_camera(
    direction = [-0.821, 0.202, -0.533],
    origin = [-10.060, 110.020, 61.037],
    up = [0.178, 0.979, 0.098]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'surfactant_details' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' cell Membrane '''
status = mm.set_camera(
    direction = [-1.0, 0.0, 0.0],
    origin = [150.0, -160, 100],
    up = [0.0, 1.0, 0.0]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'cell_membrane' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

## Movie

In [ ]:
aperture_ratio = 1.0
cameras_key_frames = [
    { # Virus overview
        'apertureRadius': aperture_ratio * 0.02,
        'focusDistance': 170.54,
        'direction': [-0.09483077240096455, -0.010968172825250144, -0.9954329830735528],
        'origin': [211.94413118453616, 21.10079920431558, 52.46345286820306],
        'up': [0.007603691659569742, 0.9999021523507218, -0.011741788515394657]        
    }, { # Protein S
        'apertureRadius': aperture_ratio * 0.02,
        'focusDistance': 32.0,
        'direction': [-0.0948307724009648, -0.010968172825250564, -0.9954329830735527],
        'origin': [211.58519458544956, 74.49932169689056, -106.41612758995753],
        'up': [0.007603691659569706, 0.9999021523507218, -0.01174178851539508]
    }, { # Protein M et E
        'apertureRadius': aperture_ratio * 0.02,
        'focusDistance': 30.33,
        'direction': [0.335260685896729, -0.0495384320761152, -0.9408220959562413],
        'origin': [175.92265055806817, 38.52391294829779, -79.12134720062772],
        'up': [0.012576359454404041, 0.998762976779797, -0.04810770621046554]
    }, { # Overview SPA
        'apertureRadius': aperture_ratio * 0.02,
        'focusDistance': 160,
        'direction': [-0.4714996076768658, -0.005614694091156256, -0.8818483969315953],
        'origin': [238.16345098602685, 46.43696346532851, 372.58472309655],
        'up': [0.003995001611884731, 0.999955871095688, -0.008502695065977069]        
    }, { # Overview SPD
        'apertureRadius': aperture_ratio * 0.02,
        'focusDistance': 393.51,
        'direction': [-0.4714996076768658, -0.005614694091156256, -0.8818483969315953],
        'origin': [238.16345098602685, 46.43696346532851, 372.58472309655],
        'up': [0.003995001611884731, 0.999955871095688, -0.008502695065977069]        
    }, { # Zoom SPD
        'apertureRadius': aperture_ratio * 0.02,
        'focusDistance': 48.60,
        'direction': [-0.8211061109992552, 0.20283488545924253, -0.5335192252584856],
        'origin': [-9.82707604633258, 110.72057107874232, 60.94402791128543],
        'up': [0.1780937547672968, 0.979106660404594, 0.09814663552179895]        
    }, { # Overview scene
        'apertureRadius': aperture_ratio * 0.0,
        'focusDistance': 1.0,
        'direction': [-1.0, 0.0, -2.220446049250313e-16],
        'origin': [1008.9571707246668, 29.057131512062583, 113.28364588186702],
        'up': [0.0, 1.0, 0.0]
    }, {
        'apertureRadius': aperture_ratio * 0.0,
        'focusDistance': 1.0,
        'direction': [-1.0, 0.0, 0.0],
        'origin': [150.0, -160, 100],
        'up': [0.0, 1.0, 0.0]
    }
]

In [ ]:
key_frames = list()
for cameras_key_frame in cameras_key_frames:
    key_frames.append(cameras_key_frame)
    key_frames.append(cameras_key_frame)

In [ ]:
def render(projection, k, samples_per_pixel, start_frame=0, end_frame=0):
    import os
    from mediamaker import MovieMaker

    nb_frames = core.get_animation_parameters()['frame_count']
    export_folder = output_folder + 'sequences/' + projection + '/'

    end_frame=nb_frames
    frame_step=1
    aperture_ratio=0.0

    output_size = list()
    if projection=='perspective':
        aperture_ratio = 1.0
        output_size = [int(k*960),int(k*540)]
        core.set_camera(current='bio_explorer_perspective')
    elif projection=='fisheye':
        output_size = [int(k*1024),int(k*1024)]
        core.set_camera(current='fisheye')
    elif projection=='panoramic':
        output_size = [int(k*1024),int(k*1024)]
        core.set_camera(current='panoramic')
    elif projection=='opendeck':
        output_size = [7*2160,3840]
        core.set_camera(current='cylindric')

    print('Rendering <' + sequence_name + '> sequence in <' + projection + '> projection ' + str(output_size))
    print(str(samples_per_pixel) + ' samples per pixel')
    print(export_folder)

    command_line='mkdir -p ' + export_folder
    os.system(command_line)
    command_line='ls ' + export_folder
    if os.system(command_line) != 0:
        print('ERROR: Failed to create output folder')
        
    size = core.get_application_parameters()['viewport']
    core.set_renderer(samples_per_pixel=1, max_accum_frames=samples_per_pixel)
    core.set_application_parameters(viewport=output_size)
    core.set_application_parameters(image_stream_fps=0)

    print('exporting...')
    
    mm.export_frames(
        path=export_folder, size=output_size,
        samples_per_pixel=samples_per_pixel-1,
        start_frame=start_frame, end_frame=end_frame
    )
    
    while mm.get_export_frames_progress()['progress'] < 1.0:
        import time
        time.sleep(1)
        
    mm.cancel_frames_export()

In [ ]:
mm.build_camera_path(key_frames, 250, 150)

In [ ]:
mm.set_current_frame(600)

In [ ]:
render('perspective', 4, 64)